 # DQN Training in Maze Environment (Hyperparameterized)

In [ ]:
import maze_library
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import datetime
import random
import os
from utils import PrioritizedReplayBuffer, moving_average, plot_episode_returns, generate_param_combinations




 ## Hyperparameters

In [ ]:


hyperparams = {
    "learning_rate":[1e-5],
    "gamma":[ 0.97],
    "epsilon_decay_episodes":[ 0.9],
    "batch_size":[ 64],
    "replay_buffer_size":[ 10000],
    "episodes":[ 4000],
    "hidden_layers":[ 
        [512, 256, 128] 
    ],
    "activation":[ "relu"],
    "target_dqn_update" :[ 10],
    
}
shared_params = {
    "epsilon_start": 1.0,
    "epsilon_min": 0.01,
    "replay_buffer_size": 10000,
    "steps_per_episode": 1000,
    "target_update_freq": 500,
    "random_seed": 42,
    "maze_width": 7,
    "maze_height": 7,
    "log_every": 100,
    "q_log_every": 2000,
    "plot_every": 1100,
    "train_after": 0.2,
    "mini_explore_runs_per_episode":3,
    "mini_exploit_runs_per_episode":2,
    
}

# Constants

 ## Maze Setup

In [ ]:
np.random.seed(42)

env = maze_library.init_environment(shared_params["maze_width"], shared_params["maze_height"], mini_exploit_runs_per_episode=shared_params["mini_exploit_runs_per_episode"], mini_explore_runs_per_episode=shared_params["mini_explore_runs_per_episode"])
maze_library.make_maze_imperfect(env)

input_shape = env.input_shape()
n_outputs = env.output_shape()


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


 ## Model Definition

In [ ]:
activation_map = {
    "relu": nn.ReLU,
    "elu": nn.ELU,
    "tanh": nn.Tanh,
    "sigmoid": nn.Sigmoid,
    # Add more if needed
}

class DQNWithCNN(nn.Module):
    def __init__(self, input_size, output_size, device, hidden_layers=[64, 64, 64], activation="elu"):
        super(DQNWithCNN, self).__init__()
        self.device = device

        # Constants
        self.visual_feature_size = 175  # 5x5x7
        self.visual_channels = 7
        self.grid_size = 5

        # Compute size of non-visual input
        self.non_visual_input_size = input_size - self.visual_feature_size

        # CNN for visual features (input shape: [batch, 7, 5, 5])
        self.cnn = nn.Sequential(
            nn.Conv2d(self.visual_channels, 32, kernel_size=3, padding=1),  # output: [batch, 32, 5, 5]
            activation_map[activation](),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # output: [batch, 64, 5, 5]
            activation_map[activation](),
            nn.MaxPool2d(2),  # output: [batch, 64, 2, 2]
        )

        # Output from CNN flattened
        cnn_output_size = 64 * 2 * 2

        # FC layers for non-visual part
        fc_input_size = cnn_output_size + self.non_visual_input_size
        self.fc_layers = nn.Sequential()
        prev_size = fc_input_size
        for i, hidden_size in enumerate(hidden_layers):
            self.fc_layers.add_module(f"fc{i}", nn.Linear(prev_size, hidden_size))
            self.fc_layers.add_module(f"act{i}", activation_map[activation]())
            prev_size = hidden_size

        # Output layer
        self.fc_layers.add_module("output", nn.Linear(prev_size, output_size))

    def forward(self, x):
        # Split input into visual and non-visual components
        visual_flat = x[:, -self.visual_feature_size:]  # Last 175 features
        non_visual = x[:, :-self.visual_feature_size]   # Remaining features

        # Reshape visual input to [batch, 7, 5, 5]
        visual = visual_flat.view(-1, self.visual_channels, self.grid_size, self.grid_size)

        # Process visual features through CNN
        visual_out = self.cnn(visual)
        visual_out = visual_out.view(visual_out.size(0), -1)  # Flatten

        # Concatenate visual and non-visual features
        x_combined = torch.cat((non_visual, visual_out), dim=1)

        # Forward through FC layers
        return self.fc_layers(x_combined)


def create_double_dqn_model(hyperprm):
    model = DQNWithCNN(input_shape, n_outputs, DEVICE, hyperprm["hidden_layers"], hyperprm["activation"]).to(DEVICE)
    target_model = DQNWithCNN(input_shape, n_outputs, DEVICE, hyperprm["hidden_layers"], hyperprm["activation"]).to(DEVICE)
    target_model.load_state_dict(model.state_dict())  
    optimizer = torch.optim.NAdam(model.parameters(), lr=hyperprm["learning_rate"])  
    loss_fn = nn.MSELoss()

    return model, target_model, optimizer, loss_fn


 ## Policy and Replay Buffer

In [ ]:
def epsilon_greedy_policy(state, epsilon, model):
    if random.random() < epsilon:
        return random.randint(0, n_outputs - 1)
    with torch.no_grad():
        state_tensor = torch.tensor(state, dtype=torch.float32, device=DEVICE).unsqueeze(0)
        q_values = model(state_tensor)
        return int(torch.argmax(q_values).item())

In [ ]:
def play_one_step(env, state, epsilon,model, replay_buffer, run):
    action = epsilon_greedy_policy(state, epsilon, model)
    action_obj = maze_library.create_action(action, run)
    next_state, reward, done, truncated = env.take_action(action_obj)
    replay_buffer.append(state, action, reward, next_state, done, truncated)
    return next_state, reward, done, truncated

In [ ]:
def training_step(model,target_model ,optimizer, loss_fn, hyperprm, replay_buffer, q_value_log, losses):
    batch = replay_buffer.sample(hyperprm["batch_size"])    
    states = torch.tensor(batch["state"], dtype=torch.float32, device=DEVICE)
    actions = torch.tensor(batch["action"], dtype=torch.int64, device=DEVICE)
    rewards = torch.tensor(batch["reward"], dtype=torch.float32, device=DEVICE)
    next_states = torch.tensor(batch["next_state"], dtype=torch.float32, device=DEVICE)
    dones = torch.tensor(batch["done"], dtype=torch.float32, device=DEVICE)
    truncateds = torch.tensor(batch["truncated"], dtype=torch.float32, device=DEVICE)
    is_weights = torch.tensor(batch["weights"], dtype=torch.float32, device=DEVICE)

    with torch.no_grad():
        next_q_values = model(next_states)
        next_actions = next_q_values.argmax(dim=1)
        target_q_values = target_model(next_states)
        max_next_q = target_q_values.gather(1, next_actions.unsqueeze(1)).squeeze()
        
        terminal = torch.logical_or(dones.bool(), truncateds.bool()).float()
        target_q = rewards + (1 - terminal) * hyperprm["gamma"] * max_next_q

    # Compute current Q-values
    q_values = model(states)
    selected_q = q_values.gather(1, actions.unsqueeze(1)).squeeze()

    # TD error and loss
    loss = loss_fn(selected_q, target_q)
    loss = (is_weights * loss.pow(2)).mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Update priorities
    td_errors = selected_q - target_q
    new_priorities = td_errors.detach().abs().cpu().numpy() + 1e-5
    replay_buffer.update_priorities(batch["indices"], new_priorities)

    losses.append(loss.item())
    q_value_log.extend(selected_q.detach().cpu().numpy())


 ## Training Loop

In [ ]:
from utils import plot_losses, plot_q_histogram


def run_single_experiment(hyperprm, timestamp,run_id):
    all_episode_explore_rewards  = []
    all_episode_exploit_rewards  = []
    losses = []
    report_cards = []
    q_value_log = []
    filecount = 0
    folder_name = f"../mazeLogs/{timestamp}DoubleDQNMaze/Run{run_id}"
    os.makedirs(folder_name, exist_ok=True) 

    model,target_model, optimizer, loss_fn = create_double_dqn_model(hyperprm)
    replay_buffer = PrioritizedReplayBuffer(capacity=hyperprm["replay_buffer_size"], state_shape=(input_shape,))
    print(f"Run {run_id}, Episodes: {hyperprm['episodes']}")
    for episode in range(hyperprm["episodes"]):

        obs = env.reset_and_regenerate()
        current_episode_explore_rewards = []
        current_episode_exploit_rewards = []
        eps_decay = episode / (hyperprm["episodes"] * hyperprm["epsilon_decay_episodes"])
        epsilon = max(hyperprm["epsilon_start"] - eps_decay, hyperprm["epsilon_min"])
        for mini_episode in range((hyperprm["mini_explore_runs_per_episode"] + hyperprm["mini_exploit_runs_per_episode"])):
            obs = env.smart_reset(mini_episode)
            for _ in range(hyperprm["steps_per_episode"]):
                obs, reward, done, truncated = play_one_step(env, obs, epsilon,model, replay_buffer, mini_episode)
                if mini_episode < hyperprm["mini_explore_runs_per_episode"]:
                    current_episode_explore_rewards.append(reward)
                else:
                    current_episode_exploit_rewards.append(reward)
                
                if done or truncated:
                    break
                

        if episode > (hyperprm["episodes"] * hyperprm["train_after"]):
            training_step(model,target_model ,optimizer, loss_fn, hyperprm, replay_buffer, q_value_log, losses)        
        if episode % hyperprm["target_dqn_update"] == 0 and episode > 0:
            target_model.load_state_dict(model.state_dict())
        
        score = maze_library.get_score(env)
        report_cards.append(score)
        all_episode_explore_rewards.append(np.sum(np.array(current_episode_explore_rewards)))
        all_episode_exploit_rewards.append(np.sum(np.array(current_episode_exploit_rewards)))
        if episode % hyperprm["log_every"] == 0 and episode != 0:
            print(f"""Run {run_id}, Total steps {score.total_steps},Average Solve Score {float(f"{score.average_run_score/ score.dijkstra_shortest_path_score:.2g}") } filecount: {filecount} 
                  Explore Rewards sum: {float(f"{np.sum(np.array(current_episode_explore_rewards)):.2g}")} Exploit Rewards sum: {float(f"{np.sum(np.array(current_episode_exploit_rewards)):.2g}")} 
                  Success Rate in solving {score.success_rate_in_exploitation} Episode: {episode}""")
            with open(f'{folder_name}/doubledqn{filecount}.json', 'w') as file:
                file.write(env.to_json_python())
            
            filecount += 1
        
        
    
    plot_q_histogram(q_value_log)
    plot_losses(losses)
    
    return {
        "params": hyperprm,
        "explore_rewards": all_episode_explore_rewards,
        "exploit_rewards": all_episode_exploit_rewards,
        "score" : report_cards,
        "q_values": q_value_log,
    }


In [ ]:
results = []
combo_params = list(generate_param_combinations(hyperparams))
timestamp = datetime.datetime.now().strftime("%d-%m_%H-%M")
run_count = 0
print(f"Totals combinations = {len(combo_params)}")
for i, combo in enumerate(combo_params):
    for j in range(2):
        merged_params = {**shared_params, **combo}
        results.append(run_single_experiment(merged_params,timestamp, run_id=run_count))
        run_count += 1
        plot_episode_returns(results[-1]["explore_rewards"], results[-1]["exploit_rewards"], 10)

    

In [ ]:
plot_episode_returns(results[-1]["explore_rewards"], results[-1]["exploit_rewards"], 10)


## Evaluation

In [ ]:


steps_nested  = [r["score"] for r in results]
steps = [score.total_steps for sublist in steps_nested for score in sublist] 


# Smoothing window size
window_size = 15  # You can adjust this window size for more or less smoothing

# Determine the number of subplots (rows x columns)
n_runs = len(steps_nested)
cols = 3  # Number of columns
rows = (n_runs // cols) + (n_runs % cols > 0)  # Calculate number of rows required

# Create a figure and axes for the subplots
fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 5))

# Flatten axes array for easy indexing
axes = axes.flatten()

# Plot each run in its own subplot
for i, run_scores in enumerate(steps_nested):
    steps = [s.total_steps for s in run_scores]
    
    # Apply moving average smoothing to the data
    smoothed_steps = moving_average(steps, window_size)
    
    # Select the current axis for this subplot
    ax = axes[i]
    
    # Plot the smoothed steps on the corresponding axis
    ax.plot(smoothed_steps, label=f"Run {i+1}")
    
    # Add labels and title
    ax.set_xlabel("Episode")
    ax.set_ylabel("Total Steps")
    ax.set_title(f"Performance of Run {i+1}")
    ax.grid(True)
    ax.legend()

# Remove any empty subplots if the number of runs isn't a perfect multiple of columns
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()